<a href="https://colab.research.google.com/github/sachinbluechip/RAG-Q-A-with-Gemini-Pro-Google-/blob/main/RAG_Q%26A_with_Gemini_Pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install -q --upgrade google-generativeai langchain-google-genai chromadb pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.0/384.0 kB 24.0 MB/s eta 0

In [2]:
from IPython.display import display
from IPython.display import Markdown
import textwrap


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [3]:
import google.generativeai as genai
from google.colab import userdata

In [4]:
import os
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

In [5]:
model = genai.GenerativeModel(model_name = "gemini-pro")
model


genai.GenerativeModel(
    model_name='models/gemini-pro',
    generation_config={},
    safety_settings={},
    tools=None,
    system_instruction=None,
    cached_content=None
)

In [6]:
response = model.generate_content("What are the usecases of LLMs?")

In [7]:

to_markdown(response.text)

> 1. **Language Generation:** LLMs can generate text that is indistinguishable from human-written text. This can be used for a variety of purposes, such as creating content for websites, social media, and marketing materials.
> 2. **Language Translation:** LLMs can translate text from one language to another. This can be used for a variety of purposes, such as translating documents, websites, and social media posts.
> 3. **Language Summarization:** LLMs can summarize text into a shorter, more concise form. This can be used for a variety of purposes, such as summarizing news articles, blog posts, and research papers.
> 4. **Question Answering:** LLMs can answer questions based on the text that they have been trained on. This can be used for a variety of purposes, such as providing customer support, answering trivia questions, and helping students with their homework.
> 5. **Code Generation:** LLMs can generate code in a variety of programming languages. This can be used for a variety of purposes, such as developing new software applications, fixing bugs, and automating tasks.
> 6. **Image Captioning:** LLMs can generate captions for images. This can be used for a variety of purposes, such as describing images for social media, creating image galleries, and helping people with visual impairments.
> 7. **Video Captioning:** LLMs can generate captions for videos. This can be used for a variety of purposes, such as describing videos for social media, creating video galleries, and helping people with hearing impairments.
> 8. **Music Generation:** LLMs can generate music. This can be used for a variety of purposes, such as creating new songs, soundtracks, and jingles.
> 9. **Medical Diagnosis:** LLMs can help doctors diagnose diseases by analyzing patient data. This can be used to identify diseases early on, which can lead to better treatment outcomes.
> 10. **Financial Analysis:** LLMs can help financial analysts make better decisions by analyzing financial data. This can be used to identify investment opportunities, predict stock prices, and manage risk.

## Chat with Documents using RAG (Retreival Augment Generation)

In [ ]:

!sudo apt -y -qq install tesseract-ocr libtesseract-dev

!sudo apt-get -y -qq install poppler-utils libxml2-dev libxslt1-dev antiword unrtf poppler-utils pstotext tesseract-ocr flac ffmpeg lame libmad0 libsox-fmt-mp3 sox libjpeg-dev swig

!pip install langchain

In [ ]:
!pip install -U langchain-community

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
import urllib
import warnings
from pathlib import Path as p
from pprint import pprint

import pandas as pd
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA



warnings.filterwarnings("ignore")
# restart python kernal if issues with langchain import.

In [12]:
from langchain_google_genai import ChatGoogleGenerativeAI

In [13]:
model = ChatGoogleGenerativeAI(model="gemini-pro",google_api_key=GOOGLE_API_KEY,
                             temperature=0.2,convert_system_message_to_human=True)

In [14]:
#Extract text from the PDF
pdf_loader = PyPDFLoader("/content/drive/MyDrive/RAG_documents/attention.pdf")
pages = pdf_loader.load_and_split()
print(pages[3].page_content)

Figure 1: The Transformer - model architecture.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively.
3.1 Encoder and Decoder Stacks
Encoder: The encoder is composed of a stack of N= 6 identical layers. Each layer has two
sub-layers. The first is a multi-head self-attention mechanism, and the second is a simple, position-
wise fully connected feed-forward network. We employ a residual connection [ 11] around each of
the two sub-layers, followed by layer normalization [ 1]. That is, the output of each sub-layer is
LayerNorm( x+ Sublayer( x)), where Sublayer( x)is the function implemented by the sub-layer
itself. To facilitate these residual connections, all sub-layers in the model, as well as the embedding
layers, produce outputs of dimension dmodel = 512 .
Decoder: The decoder is also composed of a stack of N= 6identical layers.

In [15]:
len(pages)

16

In [16]:
#RAG Pipeline: Embedding + Gemini (LLM)

from langchain_google_genai import GoogleGenerativeAIEmbeddings

text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
context = "\n\n".join(str(p.page_content) for p in pages)
texts = text_splitter.split_text(context)


embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)


vector_index = Chroma.from_texts(texts, embeddings).as_retriever(search_kwargs={"k":5})


qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True

)

In [17]:
print(texts)

['Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.comAidan N. Gomez∗ †\nUniversity of Toronto\naidan@cs.toronto.eduŁukasz Kaiser∗\nGoogle Brain\nlukaszkaiser@google.com\nIllia Polosukhin∗ ‡\nillia.polosukhin@gmail.com\nAbstract\nThe dominant sequence transduction models are based on complex recurrent or\nconvolutional neural networks that include an encoder and a decoder. The best\nperforming models also connect the encoder and decoder through an attention\nmechanism. We propose a new simple network architecture, the Transformer,\nbased solely on attention mechanisms, dispensing with recurrence and co

In [18]:
question = "Describe the Multi-head attention layer in detail?"
result = qa_chain({"query": question})
result["result"]

'The Multi-Head Attention layer is a key component of the Transformer architecture. It allows the model to attend to different parts of the input sequence simultaneously, and to learn different representations of the input.\n\nThe Multi-Head Attention layer consists of several attention heads, each of which is a self-attention mechanism. Each attention head computes a weighted sum of the values in the input sequence, where the weights are determined by a compatibility function between the query and key vectors. The output of the Multi-Head Attention layer is the concatenation of the outputs of the individual attention heads.\n\nThe Multi-Head Attention layer is used in both the encoder and decoder of the Transformer. In the encoder, the Multi-Head Attention layer allows the model to attend to different parts of the input sequence and to learn different representations of the input. In the decoder, the Multi-Head Attention layer allows the model to attend to both the input sequence and 

In [19]:

Markdown(result["result"])

The Multi-Head Attention layer is a key component of the Transformer architecture. It allows the model to attend to different parts of the input sequence simultaneously, and to learn different representations of the input.

The Multi-Head Attention layer consists of several attention heads, each of which is a self-attention mechanism. Each attention head computes a weighted sum of the values in the input sequence, where the weights are determined by a compatibility function between the query and key vectors. The output of the Multi-Head Attention layer is the concatenation of the outputs of the individual attention heads.

The Multi-Head Attention layer is used in both the encoder and decoder of the Transformer. In the encoder, the Multi-Head Attention layer allows the model to attend to different parts of the input sequence and to learn different representations of the input. In the decoder, the Multi-Head Attention layer allows the model to attend to both the input sequence and the output sequence, and to learn different representations of the output sequence.

The Multi-Head Attention layer is a powerful tool for learning representations of sequences. It allows the model to attend to different parts of the sequence simultaneously, and to learn different representations of the sequence. This makes the Multi-Head Attention layer well-suited for tasks such as machine translation and natural language processing.

In [20]:

template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)# Run chain
qa_chain = RetrievalQA.from_chain_type(
    model,
    retriever=vector_index,
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)


In [21]:

question = "Describe the Multi-head attention layer in detail?"
result = qa_chain({"query": question})
result["result"]

'Multi-Head Attention consists of several attention layers running in parallel. Each attention layer is a scaled dot-product attention function, which computes the compatibility between a query and a set of key-value pairs. The output of each attention layer is a weighted sum of the values, where the weight assigned to each value is computed by the compatibility function. The outputs of the different attention layers are then concatenated and projected to obtain the final output.\nMulti-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.\nMultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO\nwhere head i= Attention( QWQ\ni, KWK\ni, V WV\ni)\nWhere the projections are parameter matrices WQ\ni∈Rdmodel×dk,WK\ni∈Rdmodel×dk,WV\ni∈Rdmodel×dv\nandWO∈Rhdv×dmodel.\nIn this work we employ h= 8 parallel attention layers, or heads. For each of these we use\ndk=dv=dmode

In [22]:
Markdown(result["result"])

Multi-Head Attention consists of several attention layers running in parallel. Each attention layer is a scaled dot-product attention function, which computes the compatibility between a query and a set of key-value pairs. The output of each attention layer is a weighted sum of the values, where the weight assigned to each value is computed by the compatibility function. The outputs of the different attention layers are then concatenated and projected to obtain the final output.
Multi-head attention allows the model to jointly attend to information from different representation subspaces at different positions. With a single attention head, averaging inhibits this.
MultiHead( Q, K, V ) = Concat(head 1, ...,head h)WO
where head i= Attention( QWQ
i, KWK
i, V WV
i)
Where the projections are parameter matrices WQ
i∈Rdmodel×dk,WK
i∈Rdmodel×dk,WV
i∈Rdmodel×dv
andWO∈Rhdv×dmodel.
In this work we employ h= 8 parallel attention layers, or heads. For each of these we use
dk=dv=dmodel/h= 64 . Due to the reduced dimension of each head, the total computational cost
is similar to that of single-head attention with full dimensionality.
Thanks for asking!

In [23]:
question = "Describe Random forest?"
result = qa_chain({"query": question})
Markdown(result["result"])

Random forest is a supervised learning algorithm that builds an ensemble of decision trees and outputs the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random forest corrects for decision trees' habit of overfitting to their training set.

In [ ]:
!pip install gradio

In [ ]:
!!pip install pymupdf

In [ ]:
!pip install llama-index
!pip install llama-index-llms-langchain
!pip install llama-index-embeddings-langchain

In [ ]:
from llama_index.core import VectorStoreIndex, Document, ServiceContext, PromptTemplate
import gradio as gr
import fitz
import pymupdf


def read_pdf(file_path):
    #Read the content of the PDF file
    pdf_document = fitz.open(file_path)
    content = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        content += page.get_text()
    return content

    # pdf_loader = PyPDFLoader(file_path)
    # pages = pdf_loader.load_and_split()

    # return pages


def process_documents_and_answer_question(doc_files, question):
    # Read documents from uploaded files
    documents = []
    for doc_file in doc_files:
        if doc_file.name.endswith('.pdf'):
            content = read_pdf(doc_file)
        else:
            content = doc_file.read().decode("utf-8")
        documents.append(Document(text=content, metadata={"filename": doc_file.name}))
        # text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=1000)
        # context = "\n\n".join(str(p.page_content) for p in content)
        # texts = text_splitter.split_text(context)

    # with pymupdf.open(doc_files) as doc:
    #   documents = chr(12).join([page.get_text() for page in doc])

    #documents = read_pdf(doc_files)

    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001",google_api_key=GOOGLE_API_KEY)


    service_context=ServiceContext.from_defaults(
        chunk_size=1024,
        llm=model,
        embed_model=embeddings
    )

    # Build index
    index = VectorStoreIndex.from_documents(documents, service_context=service_context)

    # Query the index
    query_engine = index.as_query_engine(similarity_top_k=3)
    response = query_engine.query(question)

    return response

# Gradio Interface
interface = gr.Interface(
    fn=process_documents_and_answer_question,
    inputs=[
        gr.File(file_count="multiple", file_types=[".pdf", ".txt", ".docx"]),
        gr.Textbox(lines=3, placeholder="Enter your question here...")
    ],
    outputs="text",
    title="Document Q&A Assistant",
    description="Upload your documents and ask questions about their content."
)
#interface.clear()
# Launch the Gradio interface
interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://cd03c55d380a1d16d3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
